In [1]:
#import necessary packages
import pandas as pd
import numpy as np

# Pandas_profiling is not necessary but invaluable
import pandas_profiling

# this notebook only requires plotly express
import plotly.express as px

# for functionality
import os

In [ ]:
folderpath = r'C:\Users\ejmason\JupyterNotebooks\NPMRDS\NPMRDS_2017_TrucksOnly'
tttr2017 = r'NPMRDS_2017_TrucksOnly.csv'
os.path.join(folderpath, tttr2017)

In [21]:
# Dataset is not available to public, it requires a federally sponsored login to NPMRDS
# please see .csv for Alaska NPMRDS dataset, courtesy of Alaska Department of Transportation & Public Facilities

folderRoot = r'C:\Users\ejmason\JupyterNotebooks\NPMRDS\NPMRDS_2017_TrucksOnly'

tttr2017 = r'NPMRDS_2017_TrucksOnly.csv'
tttr2019 = r'C:\Users\ejmason\JupyterNotebooks\NPMRDS\Alaska_NPMRDS_All_TRUCKS_2019\Alaska_NPMRDS_All.csv'
tttr2017_minutes = r'C:\Users\ejmason\JupyterNotebooks\NPMRDS\NPMRDS_AK_2017_TrucksOnly_Minutes\NPMRDS_AK_2017_TrucksOnly_Minutes.csv'
tttr2019_minutes = r'C:\Users\ejmason\JupyterNotebooks\NPMRDS\NPMRDS_AK_2019_TrucksOnly_Minutes\NPMRDS_AK_2019_TrucksOnly_Minutes.csv'

def csv_to_df(data):
    data = os.path.join(folderRoot, data)
    
    print('getting data')
    print('creating dataframe')
    df = pd.read_csv(data)
    print('shape: ' + str(df.shape))
    df.sample(15)
    return (df)

df = csv_to_df(tttr2019_minutes)

getting data
creating dataframe
shape: (1124016, 7)


In [22]:
df.head()

,tmc_code,measurement_tstamp,speed,average_speed,reference_speed,travel_time_minutes,data_density
0,133-04199,2019-01-01 18:15:00,55.0,62.0,60.0,2.01,A
1,133-04199,2019-01-01 18:20:00,55.0,62.0,60.0,2.01,A
2,133-04198,2019-01-01 18:25:00,46.0,64.0,66.0,9.23,A
3,133-04191,2019-01-01 20:35:00,50.0,52.0,60.0,2.88,A
4,133-04191,2019-01-01 20:40:00,55.0,52.0,60.0,2.62,A


In [23]:
print('datetime transform')
df['measurement_tstamp'] = pd.to_datetime(df['measurement_tstamp'])

print('indexing datetime')
df.index = df['measurement_tstamp']

# some of the visualizations are greatly enhanced by some user functionality that work better with the datetime values parsed out
print('adding month/day columns')
df['month'] = df['measurement_tstamp'].dt.month
df['day_of_week'] = df['measurement_tstamp'].dt.dayofweek
df['day_name'] = df['measurement_tstamp'].dt.day_name()
df['hour'] = df['measurement_tstamp'].dt.hour

datetime transform
indexing datetime
adding month/day columns


### Select only weekdays

In [24]:
df_day = df.loc[df['day_name'].isin(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'])]

df_day.day_name.value_counts()

Monday       183335
Friday       182590
Tuesday      182011
Wednesday    177562
Thursday     176418
Name: day_name, dtype: int64

In [25]:
# # Utility class to read shapefiles
# class ShapeData:
#     def __init__(self):
#         self.data = None

#     def read_shapefile(self, shp_path):
#         """
#         Read a shapefile into a Pandas dataframe with a 'polyline' column holding
#         the geometry information. This uses the pyshp package
#         Credit: https://gist.github.com/aerispaha/f098916ac041c286ae92d037ba5c37ba
#         """
#         import shapefile

#         # read file, parse out the records and shapes
#         sf = shapefile.Reader(shp_path)
#         fields = [x[0] for x in sf.fields][1:]
#         records = sf.records()
#         shps = [s.points for s in sf.shapes()]

#         # write into a dataframe
#         self.data = pd.DataFrame(columns=fields, data=records)
#         self.data = self.data.assign(polyline=shps)
        
# # Paths to shapefile and data
# path_to_shapefile = 'C:\Users\ejmason\JupyterNotebooks\NPMRDS\NPMRDS_2017_TrucksOnly_Shapefile\Alaska.shp'
# sd = ShapeData()
# sd.read_shapefile(path_to_shapefile)


## Unreliability

# $U = TTI * T$
###### <i>normalized by miles</i>

In [11]:
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

In [12]:
df_day.groupby('tmc_code')['travel_time_minutes'].agg([percentile(10), 'mean', 'count'])

,percentile_10,mean,count
tmc_code,,,
133+04098,4.630,5.347628,1463
133+04099,11.370,14.118541,2708
133+04100,2.500,2.890553,796
133+04101,0.230,0.274675,154
133+04102,0.530,0.631639,122
...,...,...,...
133P04937,0.100,0.155807,3759
133P04938,0.070,0.085298,4360
133P04943,0.140,0.170882,1769


In [30]:
q95 = df_day.groupby('tmc_code')['travel_time_minutes'].quantile(0.95)
q50 = df_day.groupby('tmc_code')['travel_time_minutes'].quantile(0.5)

tti = q95/q50

tti = pd.DataFrame({'Tmc':tti.index, 'tti':tti.values, 'q95':q95, 'q50':q50})
# Alaska_TableToExcel - tmc data 2019 all year
df_tmc = pd.read_csv(r'C:\Users\ejmason\JupyterNotebooks\NPMRDS\Alaska_2017_TableToExcel.csv')

df_proc = df_tmc.merge(tti, on='Tmc')

df_proc['SU_CU'] = df_proc['AADT_Singl']+df_proc['AADT_Combi']
df_proc['unreliability'] = (df_proc['SU_CU'] * df_proc['tti'])/df_proc['Miles']
df_proc['unreliability_not_normalized'] = (df_proc['SU_CU'] * df_proc['tti'])

df_proc

,FID,Tmc,TmcType,RoadNumber,RoadName,IsPrimary,FirstName,TmcLinear,Country,State,...,NHS_Pct,Strhnt_Typ,Strhnt_Pct,Truck,tti,q95,q50,SU_CU,unreliability,unreliability_not_normalized
0,0,133+04261,P1,,Lake Otis Pkwy,1,E Tudor Rd,78,United States,Alaska,...,100,0,0,0,2.000000,4.2400,2.120,622,1261.237002,1244.000000
1,1,133-04819,P3,1,Seward Hwy,1,Anchorage--Kenai Peninsula County Border,65,United States,Alaska,...,100,1,100,0,2.889273,8.3500,2.890,585,631.450039,1690.224913
2,3,133-04531,P1,,E Palmer Wasilla Hwy,1,Knik Goose Bay Rd/W Riley Ave,141,United States,Alaska,...,100,0,0,0,3.109626,5.8150,1.870,763,1864.454193,2372.644385
3,4,133+04844,P1,3,W Parks Hwy,1,Puppy Haven Dr,131,United States,Alaska,...,100,1,100,1,1.188503,8.8900,7.480,639,105.053230,759.453209
4,7,133+04392,P1,,E Northern Lights Blvd,1,Boniface Pkwy,110,United States,Alaska,...,100,0,0,0,1.841270,3.4800,1.890,1620,2722.559837,2982.857143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
958,1196,133+04855,P1,3,Parks Hwy,1,Stampede Rd/Lignite Rd,131,United States,Alaska,...,100,1,100,1,2.330709,5.9200,2.540,408,402.057764,950.929134
959,1197,133+04212,P1,1,Sterling Hwy,1,Cohoe Loop,65,United States,Alaska,...,100,0,0,0,2.814634,11.5400,4.100,332,248.736475,934.458537
960,1199,133+04912,P3,7,Glacier Hwy,1,Auke Bay Ferry Terminal,212,United States,Alaska,...,96,0,0,0,1.052673,2.6580,2.525,735,458.512689,773.714851
961,1200,133-04141,P1,,W 5th Ave,1,L St,70,United States,Alaska,...,100,0,0,0,2.172727,0.7170,0.330,213,3372.226766,462.790909


In [ ]:
fig = px.scatter(df_proc,x="Miles", y="unreliability", log_y=True,
                 color="Tmc", size='tti')
fig

## Transportation Delay

# $TD = \frac{D*T}{L}$


#### NPMRDS to estimate free flow travel times (10th percentile) and average travel times for the entire year, in hours. Compared the two to estimate average annual delay per truck.

In [31]:
# D - Delay is the mean subtracted by the 10th
q10 = df.groupby('tmc_code')['travel_time_minutes'].quantile(0.1)
q10 = q10.rename('q10')

tmc_mean = df.groupby('tmc_code')['travel_time_minutes'].mean()
tmc_mean = tmc_mean.rename('tmc_mean')
df_d = pd.concat([q10,tmc_mean], axis=1).reset_index()
df_d

# merge
df_proc = df_proc.merge(df_d, left_on='Tmc', right_on='tmc_code')
df_proc

df_proc['D_delay'] = df_proc['tmc_mean'] - df_proc['q10']
df_proc['D_delay'] = df_proc['D_delay'] 
# turn seconds into hours
df_proc['trans_delay'] = df_proc['D_delay']*(df_proc['SU_CU']*365) / (60*df_proc['Miles'])

## VMT


In [32]:
df_proc['VMT'] = df_proc['AADT']*df_proc['Miles']

# View Results

In [ ]:
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.8f}'.format
df_proc.describe()

In [ ]:
df_proc.loc[df_proc['Tmc']=='133N04169']


In [ ]:
missing_tmc = '133P04641'
missing_tmc in df_proc['Tmc']

In [ ]:
df.loc[df['tmc_code']=='133N04169']['travel_time_seconds'].describe()

In [ ]:
import seaborn as sns
df_proc_r = df_proc.sort_values('VMT', ascending=False)
cmap = cmap=sns.diverging_palette(250, 5, as_cmap=True)
df_proc_r.style.background_gradient(cmap, axis=1,subset=['truck_delay', 'unreliability', 'VMT'])

# Export results

In [33]:

df_proc.to_csv(r'C:\Users\ejmason\Documents\Working Folder\freight_bottleneck_2019_minutes.csv')

In [ ]:
# Travel Time Relability Metrics (TTR)
''' EXAMPLE: family = np.where((df['Hour'] + df['Parch']) >= 1 , 'Has Family', 'No Family')
df.groupby(family)['Survived'].mean()'''
'''np.where(np.logical_and(a>=6, a<=10))'''
lottr_amp_tf = pd.Series(df.loc[np.where(np.logical_and(df['hour'].between(6,
                                                                 9, 
                                                                 inclusive=True),
                                              df['day_name'].isin(['Monday',                                    
                                                                   'Tuesday',                                
                                                                   'Wednesday',                               
                                                                   'Thursday',                  
                                                                   'Friday'])])))
lottr_amp_tf



In [ ]:
lottr_amp_tf_p80 = lottr_amp_tf.quantile(0.8)
lottr_amp_tf_p50 = lottr_amp_tf.quantile(0.5)
lottr_amp = lottr_amp_tf_p80 / lottr_amp_tf_p50
lottr_amp